# KIX 4F Simulation (yokhr)

## Imports and Initializations

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib.pyplot as plt

from copy import deepcopy

from src.simfunc.simulation import Simulation  # class for simulation
from src.utils.simparam import SimParam  # class for simulation parameters
from src.utils.utils import day_graph, minutes_to_hms

%matplotlib inline
%config InlineBackend.figure_format='retina'

## Based on Sim Demand of FY2023-2026 and 2030

### Preparation

In [ ]:
years = [2023, 2024, 2025, 2026, 2030]
dct_data = {
    year: pd.read_excel(
        f"/home/yokhr/projects/KAPpaxsim/data/raw/FY{year} KIX schedule forecast.xlsx",
        sheet_name="ALLdata",
        header=1,
    )
    for year in years
}

dct_data_processed = {}
dct_simparam = {}

# table should be formatted with following columns
# | A/D | T1/T2(MM/9C/7C/TW) | Intl Regions | Category(P/C/O) | Sector |
# | Flight Number | SEATS FC | PAX_SUM FC | Flight Date | Scheduled Time |
for key, data in dct_data.items():
    data["T1/T2(MM/9C/7C/TW)"] = data["Terminal"].replace(
        {"N": "T1", "S": "T1", 1: "T1", "L": "T2", np.nan: "-"}
    )

    data["Category(P/C/O)"] = data["Flight Type"].apply(lambda x: x[0])

    dct_name_change = {
        "Int / Dom": "Sector",
        "Int P Region": "Intl Regions",
        "Fight Number": "Flight Number",
        "Seats": "SEATS FC",
        "PAX": "PAX_SUM FC",
        "Date": "Flight Date",
        "Time": "Scheduled Time",
    }

    dct_data_processed[key] = data.rename(columns=dct_name_change)

for key, data_processed in dct_data_processed.items():
    dct_simparam[key] = SimParam()
    dct_simparam[key].schedule_from_df(
        data_processed
    ).schedule_cleanup().schedule_filter(date_str=f"{key}-12-20").assign_check_in()

### With CU for airlines using many counters during peak hours

In [ ]:
loc_start = 80
loc_end = 200
Sim_years = [2023, 2024, 2025, 2026, 2030]
CU_AL_Nms = [5, 10, 12, 15, 20, 25]
RNm_kiosk = 92  # Resource number of Kiosk
RNm_CUSBD = 32  # Resource number of CUSBD
Ptm_kiosk = 70  # Process time of Kiosk
Ptm_CUSBD = 60  # Process time of CUSBD
Ptm_CkIn = 80  # Process time of CheckIn
HrSTD_CUSBD = 2.5  # Hours to STD of CUSBD
HrSTD_CkIn = 2.5  # Hours to STD of CheckIn

for year in Sim_years:
    for CU_AL_Nm in CU_AL_Nms:
        simparam = dct_simparam[year]
        simparam_tradi = deepcopy(simparam)

        # simparam.df_Counters["total"].plot()
        # simparam.df_Counters["total"].loc[loc_start:loc_end,].plot()

        all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
        airlines_CU = (
            simparam.df_Counters.loc[loc_start:loc_end,]
            .sum(axis=0)
            .sort_values(ascending=False)
            .index.to_list()[1 : CU_AL_Nm + 1]
        )
        airlines_tradi = [x for x in all_airlines if x not in airlines_CU]

        mask_airlines_CU = simparam.schedule["Airline Code"].isin(airlines_CU)

        simparam_tradi.schedule = simparam_tradi.schedule[
            mask_airlines_CU == False
        ].copy()
        simparam_tradi.assign_check_in()

        # Plot counter number graph
        print(f"Target year is: {year}")
        print(f"CU airlines are: {len(airlines_CU)} {airlines_CU}")
        print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
        print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")
        simparam_tradi.plot_counters(
            airlines=["total"], compare_with=simparam, legend=False
        )

        # Simulation of waiting times
        simparam.show_up_from_file().assign_flight_show_up_category_default().assign_show_up()
        simparam.dct_resource = {
            "kiosk": RNm_kiosk,
            "CUSBD": RNm_CUSBD,
        }
        simparam.dct_processes = {
            "kiosk": Ptm_kiosk,
            "CUSBD": Ptm_CUSBD,
            "checkin": Ptm_CkIn,
        }
        simparam.dct_process_sequence = {
            "tradi": [
                ["kiosk", "wait_opening", "checkin"],
                {"hour_to_std": HrSTD_CkIn},
            ],
            "CUSBD": [
                ["kiosk", "wait_opening", "CUSBD"],
                {"hour_to_std": HrSTD_CUSBD},
            ],
        }

        mask_CUSBD_pax = simparam.df_Pax["Airline"].isin(airlines_CU)
        mask_tradi = mask_CUSBD_pax == False
        simparam.df_Pax.loc[mask_CUSBD_pax, "pax_type"] = "CUSBD"
        simparam.df_Pax.loc[mask_tradi, "pax_type"] = "tradi"

        # Plot Pax number graph
        freq = "1H"
        win = 1
        ratio_sampling = pd.to_timedelta("1H") / pd.to_timedelta(freq)

        plot_CU = (
            simparam.schedule[mask_airlines_CU]
            .set_index("Scheduled Time", drop=False)["PAX_SUM FC"]
            .resample(freq)
            .agg(["sum"])
            .rolling(window=win, center=True)
            .mean()
            .dropna()
            .apply(lambda x: x * ratio_sampling)
        )

        plot_tradi = (
            simparam.schedule[(mask_airlines_CU == False)]
            .set_index("Scheduled Time", drop=False)["PAX_SUM FC"]
            .resample(freq)
            .agg(["sum"])
            .rolling(window=win, center=True)
            .mean()
            .dropna()
            .apply(lambda x: x * ratio_sampling)
        )

        plot_CU = plot_CU.reindex_like(plot_tradi).fillna(0).copy()
        fig, ax = day_graph()

        nbar = 1

        colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
        nb_bar = 1
        width_hour = pd.Timedelta("0 days 01:00:00")
        width_bar = 0.7 * width_hour / nb_bar
        width = width_bar
        x = plot_tradi.index + pd.Timedelta("0 days 01:00:00")

        label = "tradi"
        ax.bar(
            x=x,
            height=plot_tradi["sum"],
            width=width,
            align="center",
            color=colors[0],
            label=label,
        )
        ax.text(
            0.15,
            0.90,
            f"{int(plot_tradi['sum'].sum()):,} Pax {label}",
            horizontalalignment="center",
            verticalalignment="center",
            transform=ax.transAxes,
            color=colors[0],
        )

        label = "CU"
        ax.bar(
            x=x,
            height=plot_CU["sum"],
            bottom=plot_tradi["sum"],
            width=width,
            align="center",
            color=colors[1],
            label=label,
        )
        ax.text(
            0.15,
            0.95,
            f"{int(plot_CU['sum'].sum()):,} Pax {label}",
            horizontalalignment="center",
            verticalalignment="center",
            transform=ax.transAxes,
            color=colors[1],
        )

        plt.show()

        dct_simparam[year].plot_std()  # Plot total Pax for testing

        # Plot simulation of waiting times
        simulation = Simulation(simparam)
        simulation.generate_checkin().generate_pax()
        simulation.run()
        simulation.format_df_result().plot_result()

### With CU for selected airlines

In [ ]:
airlines_CU_Selected = [
    #'3K',
    #'3U',
    #'4V',
    #'5J',
    #'9C',
    "AF",
    #'AQ',
    #'AY',
    #'BA',
    #'BK',
    #'BR',
    #'BX',
    #'CA',
    #'CI',
    "CX",
    #'CZ',
    #'D7',
    "DL",
    #'DR',
    #'EK',
    #'FM',
    #'GA',
    #'GJ',
    #'GK',
    #'GS',
    #'GX',
    #'HO',
    #'HU',
    #'HX',
    #'IT',
    #'JD',
    #'JL',
    #'JQ',
    #'JX',
    #'KE',
    "KL",
    #'KN',
    #'LH',
    #'LJ',
    #'MF',
    "MH",
    #'MM',
    #'MU',
    #'NH',
    #'NQ',
    #'NS',
    #'NX',
    #'NZ',
    #'OZ',
    #'PN',
    #'PR',
    #'QF',
    #'QH',
    #'QR',
    #'RA',
    #'RS',
    #'SC',
    #'SL',
    #'SQ',
    "TG",
    #'TK',
    #'TR',
    #'TW',
    #'UA',
    #'UO',
    "VJ",
    #'VN',
    #'XJ',
    #'XX',
    #'ZG',
    #'ZH'
]
for year in years:
    simparam = dct_simparam[year]
    simparam_tradi = deepcopy(simparam)

    # simparam.df_Counters["total"].plot()
    # simparam.df_Counters["total"].loc[loc_start:loc_end,].plot()

    all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
    airlines_tradi = [x for x in all_airlines if x not in airlines_CU_Selected]
    mask_airlines_CU = simparam.schedule["Airline Code"].isin(airlines_CU_Selected)
    simparam_tradi.schedule = simparam_tradi.schedule[mask_airlines_CU == False].copy()
    simparam_tradi.assign_check_in()

    print(f"Target year is: {year}")
    print(f"CU airlines are: {len(airlines_CU_Selected)} {airlines_CU_Selected}")
    print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
    print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")
    simparam_tradi.plot_counters(
        airlines=["total"], compare_with=simparam, legend=False
    )

## Based on Terminal-1 6kPAX Demand

### Preparation

In [ ]:
simparam6k = SimParam()
path_to_6k_sched = (
    r"/home/yokhr/projects/KAPpaxsim/data/processed/Schedule "
    r"(30th terminal peak, 6000 pax)_PROCESSED_yokhr.xlsx"
    # Directly remove T2 airlines: APJ, JJA, CQH
)
simparam6k.schedule_from_path(path_to_6k_sched)
simparam6k.schedule_cleanup().schedule_filter().assign_check_in()
simparam6k.plot_std(compare_with=dct_simparam[2030])
simparam6k.plot_counters(compare_with=dct_simparam[2030])

### With CU for airlines using many counters during peak hours

In [ ]:
loc_start = 80  # 0 should be set for the start of the day
loc_end = 240  # 288 should be set for the end of the day
CU_AL_Nms = [0, 5, 10, 12, 15, 20, 30, 40]

# Use these parameters later
RNm_kiosk = 92  # Resource number of Kiosk
RNm_CUSBD = 32  # Resource number of CUSBD
Ptm_kiosk = 70  # Process time of Kiosk
Ptm_CUSBD = 60  # Process time of CUSBD
Ptm_CkIn = 80  # Process time of CheckIn
HrSTD_CUSBD = 2.5  # Hours to STD of CUSBD
HrSTD_CkIn = 2.5  # Hours to STD of CheckIn

x = []
y = []

for CU_AL_Nm in CU_AL_Nms:
    simparam = simparam6k
    simparam_tradi = deepcopy(simparam)

    simparam.df_Counters["total"].plot()
    simparam.df_Counters["total"].loc[loc_start:loc_end,].plot()

    all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
    airlines_CU_6k = (
        simparam.df_Counters.loc[loc_start:loc_end,]
        .sum(axis=0)
        .sort_values(ascending=False)
        .index.to_list()[2 : CU_AL_Nm + 2]  # to skip "NEW"
    )
    airlines_tradi = [x for x in all_airlines if x not in airlines_CU_6k]

    mask_airlines_CU_6k = simparam.schedule["Airline Code"].isin(airlines_CU_6k)

    simparam_tradi.schedule = simparam_tradi.schedule[
        mask_airlines_CU_6k == False
    ].copy()
    simparam_tradi.assign_check_in()

    print("Target year is: FY2030 with 6kPAX")
    print(f"CU airlines are: {len(airlines_CU_6k)} {airlines_CU_6k}")
    print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
    print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")
    simparam_tradi.plot_counters(
        airlines=["total"], compare_with=simparam, legend=False
    )

    x = x + [CU_AL_Nm]
    y = y + [simparam_tradi.df_Counters["total"].max()]

plt.xlabel("AL with CU")
plt.ylabel("Max tradi Ctr")
plt.xticks(x)
plt.plot(x, y, marker="o", linestyle="dotted")

### With CU for selected airlines

In [ ]:
airlines_CU_6k_selected = [
    "AAR",
    "ABL",
    "AMU",
    "ANA",
    "ANZ",
    "APJ",
    "CAL",
    "CCA",
    "CDG",
    "CEB",
    "CES",
    "CPA",
    "CQH",
    "CRK",
    "CSH",
    "CSN",
    "CSZ",
    "DAL",
    "DKH",
    "DLH",
    "ESR",
    "EVA",
    "FIN",
    "GCR",
    "GIA",
    "HAL",
    "HKE",
    "HVN",
    "JAL",
    "JJA",
    "JJP",
    "JNA",
    "JSA",
    "KAL",
    "KLM",
    "MAS",
    "MSR",
    "NEW",
    "PAL",
    "SCO",
    "SIA",
    "TAX",
    "THA",
    "TTW",
    "TWB",
    "UAE",
    "UAL",
    "VNL",
    "XAX",
]
simparam = simparam6k
simparam_tradi = deepcopy(simparam)

# simparam.df_Counters["total"].plot()
# simparam.df_Counters["total"].loc[loc_start:loc_end,].plot()

all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
airlines_tradi = [x for x in all_airlines if x not in airlines_CU_6k_selected]
mask_airlines_CU = simparam.schedule["Airline Code"].isin(airlines_CU_6k_selected)
simparam_tradi.schedule = simparam_tradi.schedule[mask_airlines_CU == False].copy()
simparam_tradi.assign_check_in()

print("Target year is: FY2030 with 6kPAX")
print(f"CU airlines are: {len(airlines_CU_6k_selected)} {airlines_CU_6k_selected}")
print(f"Tradi airlines are: {len(airlines_tradi)} {sorted(airlines_tradi)}")
print(f"Max used tradi counters: {simparam_tradi.df_Counters['total'].max()}")
simparam_tradi.plot_counters(airlines=["total"], compare_with=simparam, legend=False)